## LightGBM

LightGBM 是微软开发的 boosting 集成模型，和 XGBoost 一样是对 GBDT 的优化和高效实现，原理有一些相似之处，但它很多方面比 XGBoost 有着更为优秀的表现。官方给出的这个工具库模型的优势如下：

- 更快的训练效率
- 低内存使用
- 更高的准确率
- 支持并行化学习
- 可处理大规模数据
- 支持直接使用 category 特征

下图是一组实验数据，在这份实验中，LightGBM 比 XGBoost 快将近 公式 倍，内存占用率大约为 XGBoost 的 公式，准确率也略有提升。


## LightGBM 动机
互联网领域的算法应用，通常背后都有海量的大数据。深度学习中一系列神经网络算法，都是以 mini-batch 的方式喂数据迭代训练的，总训练数据量不受内存限制。

但我们用到的机器学习算法，比如 GBDT （参考ShowMeAI文章GBDT详解）在每一次迭代的时候，都需要遍历整个训练数据多次。

- 如果把整个训练数据一次性装进内存，会明显限制训练数据的大小。
- 如果不装进内存，反复地读写训练数据又会消耗非常大的时间。
面对工业级海量的数据，普通的 GBDT 算法无法满足需求。 LightGBM 提出的主要原因之一，就是为了解决上述大数据量级下的 GBDT 训练问题，以便工业实践中能支撑大数据量并保证效率。


## LightGBM 优点

<img src="../data/img/lightGBM_vs_xgboost.png" style="zoom:50%" />
- 基于 Histogram 的决策树算法
- 带深度限制的 Leaf-wise 的叶子生长策略
- 直方图做差加速
- 直接支持类别特征（Categorical Feature）
- Cache命中率优化
- 基于直方图的稀疏特征优化
- 多线程优化

## 决策树算法
### 1）XGBoost ：Pre-sorted算法
XGBoost 使用的是 Pre-sorted 算法，能够更精确的找到数据分隔点。

- 首先，对所有特征按数值进行预排序。
- 其次，在每次的样本分割时，用 公式 的代价找到每个特征的最优分割点。
- 最后，找到最后的特征以及分割点，将数据分裂成左右两个子节点。


这种 pre-sorting 算法能够准确找到分裂点，但是在空间和时间上有很大的开销。

- 由于需要对特征进行预排序并且需要保存排序后的索引值（为了后续快速的计算分裂点），因此内存需要训练数据的两倍。
- 在遍历每一个分割点的时候，都需要进行分裂增益的计算，消耗的代价大。

### 2）LightGBM ：直方图算法
LightGBM 使用的是直方图算法（histogram algorithm），占用的内存更低，数据分割的复杂度更低。直方图算法思想是：
- 将连续的浮点特征离散成 公式 个离散值，并构造宽度为 公式 的 公式。
- 遍历训练数据，统计每个离散值在直方图中的累计统计量。
- 在进行特征选择时，只需要根据直方图的离散值，遍历寻找最优的分割点。

遍历所有特征值(n) =》 现在遍历特征值(k) 

<img src="../data/img/LightGBM_histogram.png" style="zoom:50%" />

### 内存优化
直方图算法可以很大程度降低内存消耗，它不仅不需要额外存储预排序的结果，还可以只保存特征离散化后的值（一般用 8 位整型存储就足够了）。

<img src="../data/img/lightGBM_mem.png" style="zoom:50%" />
用 8 位整型存储，内存消耗可以降低为原来的 1/8。


### 计算量优化
应用直方图算法，计算代价也大幅降低，预排序算法每遍历一个特征值就需要计算一次分裂的增益，而直方图算法只需要计算 k 次（k 可以认为是常数），时间复杂度从O(#data * #features)直接优化到 O(k * #features)。

### 注意点
- 使用分桶 bin 替代原始数据相当于增加了正则化。
- 使用分桶 bin 意味着很多数据的细节特征丢失，相似的数据如果划分到相同的桶中，数据之间的差异就无法捕获了。
- 分桶 bin 数量决定了正则化的程度，bin 越少惩罚越严重，欠拟合风险越高。
- 因为预先设定了 bin 的范围，构建直方图时不需要对数据进行排序。
- 直方图保存「划分阈值」、「当前 bin 内样本数」、「当前 bin 内所有样本的一阶梯度和」。
- 阈值的选取是按照直方图从小到大遍历，使用了上面的一阶梯度和，目的是得到划分之后 $\Delta loss$ 最大的特征及阈值。


### 直方图算法优缺点
Histogram 算法并不是完美的。由于特征被离散化后，找到的并不是很精确的分割点，所以会对结果产生影响。但在实际的数据集上表明，离散化的分裂点对最终的精度影响并不大，甚至会好一些。原因在于 decision tree 本身就是一个弱学习器，采用 Histogram 算法会起到正则化的效果，有效地防止模型的过拟合。

时间上的开销由原来的 公式 降到 公式。由于离散化，公式 远小于#data，因此时间上有很大的提升。

Histogram 算法还可以进一步加速。一个叶子节点的 Histogram 可以直接由父节点的 Histogram 和兄弟节点的 Histogram 做差得到。一般情况下，构造 Histogram 需要遍历该叶子上的所有数据，通过该方法，只需要遍历 Histogram 的k个捅。速度提升了一倍。

<img src="../data/img/lightGBM_histogram_opti.png" style="zoom:50%" />


## 决策树生长策略

### XGBoost ：Level-wise
XGBoost 采用的是Level-wise（按层生长）策略生长的，能够同时分裂同一层的叶子，从而进行多线程优化，不容易过拟合。

但不加区分的对待同一层的叶子，带来了很多没必要的开销。因为实际上很多叶子的分裂增益较低，没必要进行搜索和分裂。

### LightGBM ：Leaf-wise
LightGBM 采用 Leaf-wise（按叶子生长）生长策略，每次从当前所有叶子中找到分裂增益最大（一般也是数据量最大）的一个叶子，然后分裂，如此循环。

<img src="../data/img/lightGBM_tree.png" style="zoom:50%" />



## 类别型特征支持

大多数机器学习工具都无法直接支持类别型特征，我们会先将其编码再做后续建模，如果使用 one-hot 这种编码方式还会降低空间和时间效率。

LightGBM 优化了对类别型特征的支持，可以直接输入类别特征，不需要额外的编码或 one-hot 公式 展开。并在决策树算法上增加了类别型特征的决策规则。


## 并行化支持与优化
LightGBM 原生支持并行学习，目前支持「特征并行」和「数据并行」的两种，LightGBM 针对这两种并行方法都做了优化。

- 特征并行：在不同机器在不同的特征集合上分别寻找最优的分割点，然后在机器间同步最优的分割点。
- 数据并行：让不同的机器先在本地构造直方图，然后进行全局的合并，最后在合并的直方图上面寻找最优分割点。